In [1]:
import pulp
import numpy as np
import pandas as pd
import requests
import bs4
import urllib.request
from bs4 import BeautifulSoup
from urllib.request import urlopen
from pprint import pprint

def select_team(expected_scores, prices, positions, teams, total_budget=100, sub_factor=0.1):
    num_players = len(expected_scores)
    model = pulp.LpProblem("Constrained value maximisation", pulp.LpMaximize)
    decisions = [
        pulp.LpVariable("x{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    captain_decisions = [
        pulp.LpVariable("y{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]
    sub_decisions = [
        pulp.LpVariable("z{}".format(i), lowBound=0, upBound=1, cat='Integer')
        for i in range(num_players)
    ]


    # objective function:
    model += sum((captain_decisions[i] + decisions[i] + sub_decisions[i]*sub_factor) * expected_scores[i]
                 for i in range(num_players)), "Objective"

    # cost constraint
    model += sum((decisions[i] + sub_decisions[i]) * prices[i] for i in range(num_players)) <= total_budget  # total cost

    # position constraints
    # 1 starting goalkeeper
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 'GK') == 1
    # 2 total goalkeepers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 'GK') == 2

    # 3-5 starting defenders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 'DEF') >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 'DEF') <= 5
    # 5 total defenders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 'DEF') == 5

    # 3-5 starting midfielders
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 'MID') >= 3
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 'MID') <= 5
    # 5 total midfielders
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 'MID') == 5

    # 1-3 starting attackers
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 'FWD') >= 1
    model += sum(decisions[i] for i in range(num_players) if positions[i] == 'FWD') <= 3
    # 3 total attackers
    model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if positions[i] == 'FWD') == 3

    # team constraint
    for team_name in np.unique(teams):
        model += sum(decisions[i] + sub_decisions[i] for i in range(num_players) if teams[i] == team_name) <= 3  # max 3 players

    model += sum(decisions) == 11  # total team size
    model += sum(captain_decisions) == 1  # 1 captain
    
    for i in range(num_players):  
        model += (decisions[i] - captain_decisions[i]) >= 0  # captain must also be on team
        model += (decisions[i] + sub_decisions[i]) <= 1  # subs must not be on team

    model.solve()
    print("Total expected score = {}".format(model.objective.value()))

    return decisions, captain_decisions, sub_decisions

In [2]:
url = 'https://fplform.com/fpl-predicted-points'
r = requests.get(url)
print(r.status_code) # hopefully it's 200, so we got something
soup = BeautifulSoup(r.text, 'html.parser')

200


In [3]:
player_data = pd.read_html(url)
player_data[0]

,✔Click to move selected players to the top,Player,forInfoClick for player info popup,Team,PosPosition,CostPlayer's current price,MeritHow good the player is in terms of likelihood to score FPL points,FormOver/under-performance over the last 4 gameweeks compared to predicted points,Prob. ofAppear-ingProbability that the player will play at all in the next match,PPGW1,...,ValNext6,PPRest OfSeasonPredicted FPL points for the rest of the season,ValueRest OfSeasonPlayer's value for the rest of the season (predicted points per million pounds excess cost),PointsSo Far,OfficialChancePlayer's official chance of playing the next match,OfficialAvailability,SelectedBy %Percentage of managers owning this player,TransfersIn GWNumber of transfers in since the last transfer deadline,TransfersOut GWNumber of transfers out since the last transfer deadline,News
0,NaN,Son,NaN,TOT,MID,12.0,6.61,1.43,0.97,6.3,...,4.11,214,25.2,258,100,Available,25.2,0,0,NaN
1,NaN,Salah,NaN,LIV,MID,13.0,6.74,0.91,0.97,6.1,...,3.77,219,23.1,265,100,Available,61.0,0,0,NaN
2,NaN,Kane,NaN,TOT,FWD,11.5,6,1.43,0.97,6.0,...,3.73,177,22.1,192,100,Available,27.7,0,0,NaN
3,NaN,Perišić,NaN,TOT,DEF,5.5,4.5,0.37,0.67,5.9,...,11.05,166,66.3,0,100,Available,28.0,0,0,NaN
4,NaN,Alexander-Arnold,NaN,LIV,DEF,7.5,5.87,0.67,0.81,5.7,...,8.5,214,47.5,208,100,Available,59.0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554,NaN,Rondón,NaN,EVE,FWD,5.0,1.45,0.59,0,0.0,...,5.04,55,37,30,0,Suspended,0.2,0,0,Suspended until 13 Aug
555,NaN,Hendrick,NaN,NEW,MID,4.5,1.7,1,0,0.0,...,0,0,0,8,0,Unavailable,0.0,0,0,Joined Reading on loan - Expected back 01 Jul
556,NaN,Roberts,NaN,BHA,DEF,4.0,1.57,1,0,0.0,...,0,0,0,0,0,Unavailable,0.0,0,0,Joined Derby County on loan - Expected back 01...
557,NaN,André Gomes,NaN,EVE,MID,4.5,1.16,0.96,0,0.0,...,5.7,50,50.4,18,0,Injured,0.4,0,0,Muscle injury - Expected back 20 Aug


In [4]:
players = player_data[0].loc[:,"Player"]
teams = player_data[0].loc[:,"Team"]
positions = player_data[0].loc[:,"PosPosition"]
prices = player_data[0].loc[:,"CostPlayer's current price"]
ppnext6 = player_data[0].loc[:,"PPNext6"]
ppall = player_data[0].loc[:,"PPRest OfSeasonPredicted FPL points for the rest of the season"]

In [5]:
# The last values of our arrays of data picked up some ugly parts of the site we scraped:

In [6]:
ppnext6.values
ppall.values
prices.values
positions.values
players.values
teams.values

array(['TOT', 'LIV', 'TOT', 'TOT', 'LIV', 'LIV', 'TOT', 'TOT', 'LIV',
       'NEW', 'TOT', 'LIV', 'TOT', 'CHE', 'TOT', 'MCI', 'CHE', 'TOT',
       'NEW', 'TOT', 'TOT', 'TOT', 'LIV', 'LEI', 'CHE', 'LIV', 'CHE',
       'NEW', 'CHE', 'LEE', 'AVL', 'AVL', 'TOT', 'NEW', 'MUN', 'NEW',
       'MCI', 'MCI', 'MCI', 'MUN', 'NEW', 'WOL', 'CHE', 'CRY', 'LIV',
       'LIV', 'CRY', 'AVL', 'NEW', 'LEI', 'MCI', 'BHA', 'CRY', 'NEW',
       'WOL', 'TOT', 'NEW', 'CHE', 'MCI', 'CHE', 'NEW', 'ARS', 'CRY',
       'AVL', 'LEI', 'TOT', 'MUN', 'ARS', 'AVL', 'NEW', 'BHA', 'WOL',
       'CRY', 'BOU', 'TOT', 'AVL', 'WOL', 'LEI', 'LEI', 'WOL', 'MCI',
       'WOL', 'WOL', 'LEE', 'WHU', 'AVL', 'ARS', 'CRY', 'MUN', 'LEE',
       'ARS', 'WHU', 'ARS', 'CHE', 'SOU', 'BOU', 'WOL', 'TOT', 'AVL',
       'BRE', 'MUN', 'ARS', 'LEE', 'WOL', 'MCI', 'TOT', 'ARS', 'LEI',
       'LIV', 'TOT', 'CHE', 'BRE', 'LIV', 'AVL', 'CRY', 'ARS', 'BHA',
       'CHE', 'LEE', 'CHE', 'MCI', 'LIV', 'BHA', 'BHA', 'NFO', 'AVL',
       'AVL', 'WOL',

In [7]:
chopped_ppnext6 = ppnext6.values[0:-1]
chopped_prices = prices.values[0:-1]
chopped_positions = positions.values[0:-1]
chopped_players = players.values[0:-1]
chopped_teams = teams.values[0:-1]
chopped_ppall = ppall.values[0:-1]

best_ppnext6 = pd.Series(chopped_ppnext6)
best_prices = pd.Series(chopped_prices)
best_positions = pd.Series(chopped_positions)
best_players = pd.Series(chopped_players)
best_teams = pd.Series(chopped_teams)
best_ppall = pd.Series(chopped_ppall)

best_teams

0      TOT
1      LIV
2      TOT
3      TOT
4      LIV
      ... 
553    MCI
554    EVE
555    NEW
556    BHA
557    EVE
Length: 558, dtype: object

In [8]:
# Team made for the close future (next 6 gameweeks):

decisions, captain_decisions, sub_decisions = select_team(best_ppnext6, best_prices, best_positions, best_teams)
player_indices = []

print()
print("First Team:")
for i in range(len(decisions)):
    if decisions[i].value() == 1:
        print("{}{}".format(best_players[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(best_players[i])
        player_indices.append(i)

C:\Users\ryvo1\anaconda3\envs\PIC16B\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Total expected score = 371.33

First Team:
Son
Kane
Perišić
Alexander-Arnold*
Robertson
Matip
Pope
Foden
Cancelo
Trossard
Ward-Prowse

Subs:
Iversen
Amad
Dervişoğlu
Taylor


In [9]:
# Team made for the entire season (set and forget, no transfers):

decisions, captain_decisions, sub_decisions = select_team(best_ppall, best_prices, best_positions, best_teams)
player_indices = []

print()
print("First Team:")
for i in range(len(decisions)):
    if decisions[i].value() == 1:
        print("{}{}".format(best_players[i], "*" if captain_decisions[i].value() == 1 else ""))
        player_indices.append(i)
print()
print("Subs:")
for i in range(len(sub_decisions)):
    if sub_decisions[i].value() == 1:
        print(best_players[i])
        player_indices.append(i)

Total expected score = 2197.0

First Team:
Son*
Perišić
Alexander-Arnold
Robertson
Sterling
Koulibaly
Alisson
Foden
Cancelo
Gündogan
Toney

Subs:
Iversen
Amad
Dervişoğlu
Plange
